In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torchvision.models as models
import time
import os
import uuid
from itertools import chain
import torchvision
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
import sys
from scipy.spatial.distance import pdist, squareform, cdist
import json
import sys
import math

In [4]:
cudan = 1

In [5]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class PreActBlock(nn.Module):
    '''Pre-activation version of the BasicBlock.'''
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(PreActBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = F.relu(self.bn1(x))
        shortcut = self.shortcut(out)
        out = self.conv1(out)
        out = self.conv2(F.relu(self.bn2(out)))
        out += shortcut
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class PreActBottleneck(nn.Module):
    '''Pre-activation version of the original Bottleneck module.'''
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(PreActBottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = F.relu(self.bn1(x))
        shortcut = self.shortcut(out)
        out = self.conv1(out)
        out = self.conv2(F.relu(self.bn2(out)))
        out = self.conv3(F.relu(self.bn3(out)))
        out += shortcut
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = conv3x3(3,64)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, lin=0, lout=5):
        out = x
        if lin < 1 and lout > -1:
            out = self.conv1(out)
            out = self.bn1(out)
            out = F.relu(out)
        if lin < 2 and lout > 0:
            out = self.layer1(out)
        if lin < 3 and lout > 1:
            out = self.layer2(out)
        if lin < 4 and lout > 2:
            out = self.layer3(out)
        if lin < 5 and lout > 3:
            out = self.layer4(out)
        if lout > 4:
            out = F.avg_pool2d(out, 4)
            out = out.view(out.size(0), -1)
            out = self.linear(out)
        return out


def ResNet18():
    return ResNet(PreActBlock, [2,2,2,2])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])



In [6]:
net = ResNet18()
import torch.backends.cudnn as cudnn
net.cuda()
net = torch.nn.DataParallel(net)
print(torch.cuda.device_count())
cudnn.benchmark = True
print('Using CUDA..')
net.load_state_dict(torch.load("./norm_ResNet18_epo_200"))

2
Using CUDA..


<All keys matched successfully>

In [7]:
device = torch.device("cuda")

net.eval()

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): PreActBlock(
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (shortcut): Sequential()
      )
      (1): PreActBlock(
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [8]:
c = -0.001
print('c: ', c)

c:  -0.001


In [10]:
import time
class LinfPGDAttack:
    def __init__(self, model, epsilon, k, a, random_start, loss_func):
        self.model = model
        self.epsilon = epsilon
        self.k = k
        self.a = a
        self.rand = random_start
        self.loss_func = loss_func
        self.step_size = 0.01
        if loss_func == 'xent':
              self.loss_fun =  torch.nn.CrossEntropyLoss()
        self.grad = None
    
    def find_grad(self,input_image, input_label):
        input_image = torch.autograd.Variable(torch.tensor(input_image))
        input_image.requires_grad= True
        input_label = torch.tensor(input_label).to(device,dtype=torch.int64)
        prediction = self.model(input_image.to(device,dtype=torch.float))
        loss_value = self.loss_fun( prediction,input_label)
        gradient = torch.autograd.grad(loss_value, input_image)
        return np.array(gradient[0])

    def perturb(self, x_nat, x_adv, y):

        batch_size = x_nat.shape[0]  
        for epoch in range(10):

            grad = self.find_grad(x_adv, y)            
            kxy, dxkxy = self.svgd_kernel(x_adv)
            svgd = -(np.matmul(kxy, -np.array(grad).reshape([-1, 32*32*3])).reshape([-1, 3, 32, 32]) + dxkxy)/batch_size
            x_adv = np.add(x_adv, self.step_size * np.sign(c*svgd + grad), out=x_adv, casting='unsafe')

            x_adv = np.clip(x_adv, x_nat - self.epsilon, x_nat + self.epsilon) 
            x_adv = np.clip(x_adv, min_clip, max_clip) 

        return x_adv

    def svgd_kernel(self, Theta):
        theta = Theta.reshape([-1, 32*32*3])

        sq_dist = pdist(theta)
        pairwise_dists = squareform(sq_dist)**2

        h = np.median(pairwise_dists)
        h = np.sqrt(0.5*h/np.log(theta.shape[0]))


        Kxy = np.exp(-pairwise_dists / h**2 / 2)

        dxkxy = -np.matmul(Kxy, theta)
        sumkxy = np.sum(Kxy, axis=1)

        for i in range(theta.shape[1]):
            dxkxy[:, i] = dxkxy[:,i] + np.multiply(theta[:,i],sumkxy)
        dxkxy = dxkxy / (h**2)

        dxkxy = np.reshape(dxkxy, [-1, 3, 32, 32])
        return (Kxy, dxkxy)

    def wgf_kernel(self, theta):
        sq_dist = pdist(theta)
        pairwise_dists = squareform(sq_dist)**2

        h = np.median(pairwise_dists)  
        h = np.sqrt(0.5 * h / np.log(theta.shape[0] + 1))

        Kxy = np.exp( -pairwise_dists / h**2 / 2)
        Kxy = np.multiply((pairwise_dists / h**2 / 2 - 1), Kxy)

        dxkxy = -np.matmul(Kxy, theta)
        sumkxy = np.sum(Kxy, axis=1)
        for i in range(theta.shape[1]):
            dxkxy[:, i] = dxkxy[:,i] + np.multiply(theta[:,i],sumkxy)

        return (Kxy, dxkxy)

In [13]:
BATCH_SIZE = 128

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


train_dataset = datasets.CIFAR10(root='../data', 
                               train=True, 
                               transform=transform_test,
                               download=True)

test_dataset = datasets.CIFAR10(root='../data', 
                              train=False, 
                              transform=transform_test)


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False)

Files already downloaded and verified


In [18]:
new_test_data = []
for x,y in test_dataset:
    new_test_data.append(x.view(-1,3,32,32))

In [19]:
new_test_data = torch.cat(new_test_data, dim=0)

In [21]:
max_clip = torch.max(new_test_data).item()
min_clip = torch.min(new_test_data).item()

In [27]:
def make_categorical(arr,num_class):
    cat_arr = np.zeros((len(arr),num_class))
    for k in range(len(arr)):
        cat_arr[k][arr[k]] = 1
    return(cat_arr)

In [28]:
test_data = np.array(new_test_data.data)
test_tar = np.array(test_dataset.targets)
print(test_tar.shape)
test_lab_categorical = make_categorical(np.array(test_dataset.targets),10)

(10000,)


In [29]:
attack = LinfPGDAttack(net,
                         0.20,
                         100,
                         0.01,
                         True,
                         "xent")
x_adv_final = np.copy(test_data)
y_adv_final = np.copy(test_tar)
num_eval_examples = len(test_data) 
eval_batch_size = 64
num_batches = int(len(test_data)/eval_batch_size)

In [30]:
for restart in range(10):
    succ = 0
    permutation = np.arange(num_eval_examples)
    idx = np.arange(num_eval_examples)
    print("min",torch.min(torch.tensor(test_data)))
    print("max",torch.max(torch.tensor(test_data)))
    x_test, y_test = np.copy(test_data), np.copy(test_tar)

    x_adv = x_test + np.random.uniform(-attack.epsilon, attack.epsilon, x_test.shape)
    print("min",torch.min(torch.tensor(x_adv)))
    print("max",torch.max(torch.tensor(x_adv)))


    t0 = time.time()
    x_test_temp, x_adv_temp, y_test_temp = np.copy(test_data), np.copy(x_adv), np.copy(test_tar)
    for epoch in range(int(attack.k/10)):
        np.random.shuffle(idx)
        x_test_temp, x_adv_temp, y_test_temp = x_test_temp[idx], x_adv_temp[idx], y_test_temp[idx]
        permutation = permutation[idx]

        for ibatch in range(num_batches):
            print("====> restart : {}\{} ====> epoch : {}\{} ====> ibatch : {}\{}".format(restart,10,epoch,int(attack.k/10),ibatch,num_batches))
            sys.stdout.flush()
            bstart = ibatch * eval_batch_size
            bend = min(bstart + eval_batch_size, num_eval_examples)

            x_batch = x_test_temp[bstart:bend, :]
            x_batch_adv = x_adv_temp[bstart:bend, :]
            y_batch = y_test_temp[bstart:bend]

            x_adv_temp[bstart:bend, :] = attack.perturb(x_batch, x_batch_adv, y_batch)

    inv_permutation = np.argsort(permutation)
    x_adv = x_adv_temp[inv_permutation]
    print("x_adv.shape",x_adv.shape)
    print("test_data.shape",test_data.shape)
    print('round Linf: ', np.max(np.abs(x_adv - test_data)))

    sys.stdout.flush()

    for ibatch in range(num_batches):
        bstart = ibatch * eval_batch_size
        bend = min(bstart + eval_batch_size, num_eval_examples)

        x_batch_adv = x_adv[bstart:bend, :]
        y_batch = y_test[bstart:bend]

        y_pred = attack.model(torch.tensor(x_batch_adv).to(device,dtype=torch.float))#,training=False)

        y_pred = np.argmax(y_pred.detach().cpu().numpy(), 1)


        correct_prediction = np.equal(y_pred, y_batch)

        for i in range(correct_prediction.shape[0]):
            if not correct_prediction[i]:
                succ+=1
                x_adv_final[bstart+i] = x_batch_adv[i] 
                y_adv_final[bstart+i] = y_batch[i] 
                
    print("Success : ",succ)
    print("Time for repeat: {}".format((time.time()-t0)/60))
    sys.stdout.flush()
    

min tensor(0.)
max tensor(1.)
min tensor(-0.2000, dtype=torch.float64)
max tensor(1.2000, dtype=torch.float64)
====> restart : 0\10 ====> epoch : 0\10 ====> ibatch : 0\156
====> restart : 0\10 ====> epoch : 0\10 ====> ibatch : 1\156
====> restart : 0\10 ====> epoch : 0\10 ====> ibatch : 2\156
====> restart : 0\10 ====> epoch : 0\10 ====> ibatch : 3\156
====> restart : 0\10 ====> epoch : 0\10 ====> ibatch : 4\156
====> restart : 0\10 ====> epoch : 0\10 ====> ibatch : 5\156
====> restart : 0\10 ====> epoch : 0\10 ====> ibatch : 6\156
====> restart : 0\10 ====> epoch : 0\10 ====> ibatch : 7\156
====> restart : 0\10 ====> epoch : 0\10 ====> ibatch : 8\156
====> restart : 0\10 ====> epoch : 0\10 ====> ibatch : 9\156
====> restart : 0\10 ====> epoch : 0\10 ====> ibatch : 10\156
====> restart : 0\10 ====> epoch : 0\10 ====> ibatch : 11\156
====> restart : 0\10 ====> epoch : 0\10 ====> ibatch : 12\156
====> restart : 0\10 ====> epoch : 0\10 ====> ibatch : 13\156
====> restart : 0\10 ====> epoc

KeyboardInterrupt: 

In [ ]:
np.save("DAA_adv_blob_test_mixup_norm_100",np.array(x_adv_final ))
np.save("DAA_adv_blob_target_mixup_norm_100",np.array(y_adv_final ))
print("adv images saved")
sys.stdout.flush()